In [1]:
import pyproj
pyproj.datadir.set_data_dir("/home/yifan/anaconda3/envs/myenv/share/proj")

/home/yifan/anaconda3/envs/myenv/lib/python3.9/site-packages/pyproj/__init__.py:95: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
print(pyproj.datadir.get_data_dir())

/home/yifan/anaconda3/envs/myenv/share/proj


In [3]:
import sys
import pyproj
import geopandas as gpd
print(sys.version)
print(pyproj.__version__)
print(gpd.__version__)
import rasterio
from rasterio import features
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, jaccard_score, confusion_matrix

3.9.18 | packaged by conda-forge | (main, Dec 23 2023, 16:33:10) 
[GCC 12.3.0]
3.6.1
0.14.2


In [7]:
forest_2020 = gpd.read_file('Zone_Dataset_Mask_2020.shp')
print(forest_2020)
forest_2021 = gpd.read_file('Zone_Dataset_Mask_2021.shp')
print(forest_2021)

        fid  DN                                           geometry
0       4.0   1  POLYGON ((-70.49385 -8.40742, -70.49358 -8.407...
1      10.0   1  POLYGON ((-70.53023 -8.40742, -70.52969 -8.407...
2      11.0   1  POLYGON ((-70.50813 -8.40742, -70.50733 -8.407...
3      17.0   1  POLYGON ((-70.49331 -8.40769, -70.49277 -8.407...
4      19.0   1  POLYGON ((-70.62483 -8.40796, -70.62456 -8.407...
..      ...  ..                                                ...
653  2926.0   1  POLYGON ((-70.56473 -8.57748, -70.56446 -8.577...
654  2928.0   1  POLYGON ((-70.65097 -8.57721, -70.65017 -8.577...
655  2930.0   1  POLYGON ((-70.64477 -8.58071, -70.64451 -8.580...
656  2931.0   1  POLYGON ((-70.54506 -8.46887, -70.54479 -8.468...
657  2938.0   1  POLYGON ((-70.55126 -8.58071, -70.55099 -8.580...

[658 rows x 3 columns]
        fid  DN                                           geometry
0       4.0   1  POLYGON ((-70.49385 -8.40742, -70.49358 -8.407...
1      10.0   1  POLYGON ((-70.53023 -

In [9]:
# 定义栅格化的分辨率
pixel_size = 0.0001  # 根据你的需求调整分辨率

# 获取两者的联合边界（bounding box）
bounds = forest_2020.total_bounds
minx, miny, maxx, maxy = bounds

# 创建输出的栅格形状
width = int((maxx - minx) / pixel_size)
height = int((maxy - miny) / pixel_size)

# 定义栅格的元数据
transform = rasterio.transform.from_bounds(minx, miny, maxx, maxy, width, height)

# 栅格化函数
def rasterize_polygon(gdf, transform, width, height):
    shapes = [(geom, 1) for geom in gdf.geometry]
    raster = features.rasterize(shapes, out_shape=(height, width), transform=transform, fill=0, dtype='int16')
    return raster

In [10]:
raster1 = rasterize_polygon(forest_2020,transform,width,height)
raster2 = rasterize_polygon(forest_2021,transform,width,height)

In [11]:
overlap = np.logical_and(raster1 == 1, raster2 == 1)
print(overlap)

[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [12]:
raster1_flat = raster1.flatten()
raster2_flat = raster2.flatten()

In [13]:
# 计算分类评估指标
precision = precision_score(raster1_flat, raster2_flat, average='binary')
recall = recall_score(raster1_flat, raster2_flat, average='binary')
f1 = f1_score(raster1_flat, raster2_flat, average='binary')
jaccard = jaccard_score(raster1_flat, raster2_flat, average='binary')

In [14]:
# 输出结果
print(f"Chevauchement: {np.sum(overlap)}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"Jaccard Coefficient: {jaccard}")

Chevauchement: 2190192
Precision: 1.0
Recall: 0.975772229283699
F1-Score: 0.9877375689579944
Jaccard Coefficient: 0.975772229283699
